In [1]:
import os
import pickle

import keras.utils
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications import Xception
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Activation, Flatten, Dense
from keras.layers import (concatenate)
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
from skimage.transform import resize
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

train_image = os.path.join(data_dir, "train_images")
test_image = os.path.join(data_dir, "test_images")

label_column = "AdoptionSpeed"

BATCH_SIZE, test_size = 512, 0.2

height, width = 100, 100

Using TensorFlow backend.


In [2]:
def read_csv_kaggle(path, is_train):
    train = pd.read_csv(path, sep=',')
    pet_ids = train["PetID"]

    selected_columns = ["Type",
                        "Gender",
                        "Color1",
                        "Color2",
                        "Color3",
                        "MaturitySize",
                        "FurLength",
                        "Vaccinated",
                        "Dewormed",
                        "Sterilized",
                        "Health",
                        #"State",
                        "MaturitySize"]
    
    y = train[label_column] if is_train else None
    
    # One-Hot-encode
    X = pd.get_dummies(train[selected_columns], columns=selected_columns)

    # Normalize:
    to_normalize = ["Age", "Fee", "Quantity"]
    for to_norm in to_normalize:
         X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()
    
    return X, y, pet_ids

def read_images(image_paths):
    def random_image():
        return np.random.rand(height, width, 3) * 255
    
    def read_image(path):
        img = np.asarray(Image.open(path).convert("RGB"), dtype="int32" )
        return resize(img, (height, width), anti_aliasing=True, mode='constant')    

    return np.array([read_image(path) if os.path.isfile(path) else random_image() for path in tqdm(image_paths)])

In [3]:
X, y, pet_ids = read_csv_kaggle(os.path.join(data_dir, "train.csv"), True)

In [4]:
f_im_name = "images.binary"

if not os.path.isfile(f_im_name):
    images = read_images([os.path.join(train_image, pet_id + "-1.jpg") for pet_id in pet_ids])
    
    # Standardize:
    mean, std = np.mean(images), np.std(images)
    images_meanstd = (images - mean)/std
    
    with open(f_im_name, 'wb') as handle_1, open("mnstd", 'wb') as handle_2:
        pickle.dump(images_meanstd, handle_1, protocol=pickle.HIGHEST_PROTOCOL)        
        pickle.dump((mean, std), handle_2, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f_im_name, 'rb') as handle_1, open("mnstd", 'rb') as handle_2:
        images_meanstd = pickle.load(handle_1)
        temp = pickle.load(handle_2)
        mean, std = temp[0], temp[1]
                
print(images_meanstd.shape)

100%|████████████████████████████████████████████████████████████████████████████| 14993/14993 [08:06<00:00, 30.83it/s]


(14993, 100, 100, 3)


In [5]:
X_train_img, X_test_img, X_train_else, X_test_else, y_train, y_test = train_test_split(images_meanstd, 
                                                                                       X, 
                                                                                       y, 
                                                                                       test_size=test_size,
                                                                                       random_state=1)

In [6]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

transfer = Xception(weights='imagenet', include_top=False, input_shape=(height, width, 3))

# Freeze Xception
for layer in transfer.layers[:-3]:
    layer.trainable = False

# Inputs
image_input = Input(shape=(height, width, 3))
aux_input = Input(shape=(len(list(X_train_else)),))

# Images:
transfer = transfer(image_input)
transfer = Dense(150, activation='relu')(transfer)
flatten = Flatten()(transfer)

# Aux input:
x = Dense(150, activation='relu')(aux_input)
x = Dense(250, activation='relu')(x)
x = Dense(350, activation='relu')(x)

# Merged:
merge = concatenate([flatten, x])
x = Dense(500)(merge)
x = Dense(450, activation='relu')(x)
x = Dense(100, activation='relu')(x)
h = Dense(5)(x)

# Predictions:
predictions = Activation('softmax')(h)

model = Model(inputs=[image_input, aux_input], outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit([X_train_img, X_train_else], 
                    keras.utils.to_categorical(y_train),
                    batch_size=BATCH_SIZE, 
                    epochs=7, 
                    validation_split=0.1,
                    callbacks=[ModelCheckpoint('test_model.h5', save_best_only=True)])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 10794 samples, validate on 1200 samples
Epoch 1/7
10794/10794 [==============================] - ETA: 32:22 - loss: 1.6136 - acc: 0.25 - ETA: 29:02 - loss: 1.5903 - acc: 0.25 - ETA: 27:28 - loss: 1.5635 - acc: 0.27 - ETA: 26:19 - loss: 1.5915 - acc: 0.27 - ETA: 24:40 - loss: 1.5770 - acc: 0.27 - ETA: 22:41 - loss: 1.5794 - acc: 0.28 - ETA: 20:28 - loss: 1.5642 - acc: 0.28 - ETA: 18:29 - loss: 1.5712 - acc: 0.28 - ETA: 16:56 - loss: 1.5709 - acc: 0.28 - ETA: 15:15 - loss: 1.5617 - acc: 0.29 - ETA: 13:45 - loss: 1.5577 - acc: 0.29 - ETA: 12:06 - loss: 1.5540 - acc: 0.29 - ETA: 10:32 - loss: 1.5480 - acc: 0.30 - ETA: 9:04 - loss: 1.5440 - acc: 0.3057 - ETA: 7:39 - loss: 1.5370 - acc: 0.307 - ETA: 6:18 - loss: 1.5329 - acc: 0.310 - ETA: 5:00 - loss: 1.5296 - acc: 0.313 - ETA: 3:43 - loss: 1.5235 - acc: 0.316 - ETA: 2:29 - loss: 1.5175 - acc: 0.318 - ETA: 1:17 - l

In [7]:
train_pred = [np.argmax(pred) for pred in model.predict([X_train_img, X_train_else])]
test_predictions = [np.argmax(pred) for pred in model.predict([X_test_img, X_test_else])]

In [8]:
print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

Kappa on train: 0.3381
Accuracy on train: 0.3951
________________
Kappa on test: 0.2702
Accuracy on test: 0.3451


In [9]:
test, _, test_pet_ids = read_csv_kaggle(os.path.join(data_dir, "test.csv"), False)
test_images = read_images([os.path.join(test_image, pet_id + "-1.jpg") for pet_id in test_pet_ids])
test_images_std = (test_images - mean)/std

100%|██████████████████████████████████████████████████████████████████████████████| 3972/3972 [01:55<00:00, 37.38it/s]


In [10]:
loaded = load_model('test_model.h5')
test_pred = loaded.predict([test_images_std, test])